In [36]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



In [24]:
df = pd.read_csv('data.csv')

# print(pd.head())

C:\Users\isaac\AppData\Local\Temp\ipykernel_13244\2718534721.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


In [3]:
print(df.columns.tolist())

['id', 'client_id', 'job_id', 'fid', 'title', 'date_of_posting', 'job_role', 'api_job_gender', 'year_of_passing_from', 'year_of_passing_to', 'hq_marks_percentage_from', 'hq_marks_percentage_to', 'hq_marks_cgpa_from', 'hq_marks_cgpa_to', 'api_applications_created', 'call_letter_date', 'applier_type', 'course_type', 'users_city_id', 'users_state_id', 'profile_percentage', 'no_of_employees', 'bd_lnkdin_co_profile_emp2', 'FemaleFlag', 'CalLetterFlag', 'Chennai', 'Kolkata', 'Hyderabad', 'NCR', 'Ahmedabad', 'Bangalore', 'MumbaiPune', 'India_HQ', 'napplications_t_male_jbrl', 'napplications_t_female_jbrl', 'napplicants_t_male_jbrl', 'napplicants_t_female_jbrl', 'client_id_napplications_t', 'client_id_njob_ids_t', 'njob_ids_t_jbrl', 'yearBirth', 'CashFlow', 'ProfitMargin_Percent', 'ROE_Percent', 'ROCE_Percent', 'CurrentRatio', 'SolvencyRatio', 'OperatingRevenue', 'OperatingProfitPerShare', 'EarningsPerShare', 'NetIncome', 'P_LBeforeTax', 'ShareholdersFunds', 'TotalAssets', 'MarketCap', 'PriceEa

In [5]:
pd.options.display.max_columns = None
df

,id,client_id,job_id,fid,title,date_of_posting,job_role,api_job_gender,year_of_passing_from,year_of_passing_to,hq_marks_percentage_from,hq_marks_percentage_to,hq_marks_cgpa_from,hq_marks_cgpa_to,api_applications_created,call_letter_date,applier_type,course_type,users_city_id,users_state_id,profile_percentage,no_of_employees,bd_lnkdin_co_profile_emp2,FemaleFlag,CalLetterFlag,Chennai,Kolkata,Hyderabad,NCR,Ahmedabad,Bangalore,MumbaiPune,India_HQ,napplications_t_male_jbrl,napplications_t_female_jbrl,napplicants_t_male_jbrl,napplicants_t_female_jbrl,client_id_napplications_t,client_id_njob_ids_t,njob_ids_t_jbrl,yearBirth,CashFlow,ProfitMargin_Percent,ROE_Percent,ROCE_Percent,CurrentRatio,SolvencyRatio,OperatingRevenue,OperatingProfitPerShare,EarningsPerShare,NetIncome,P_LBeforeTax,ShareholdersFunds,TotalAssets,MarketCap,PriceEarningsRatio_Avg
0,514406735,500004,198902,15552098,CLOUD ENGINEER,1464978601,13,3,2010.0,2015.0,NaN,100.0,NaN,10.0,1465836940,NaN,f,graduation,125,30,55,5.0,"10,001+ employees",0,0,1,0,0,0,0,0,0,0,112523,70630,33587,21011,81,1,1104,1994,2.828000e+06,7.797,NaN,39.631,0.982,-13.416,4.823800e+07,2.299,1.459,2.496000e+06,3.761000e+06,-3.889000e+06,2.898700e+07,24790.588,8.496
1,518735632,500004,198902,16166171,CLOUD ENGINEER,1464978601,13,3,2010.0,2015.0,NaN,100.0,NaN,10.0,1467078129,NaN,f,graduation,125,30,80,5.0,"10,001+ employees",1,0,1,0,0,0,0,0,0,0,112523,70630,33587,21011,81,1,1104,1994,2.828000e+06,7.797,NaN,39.631,0.982,-13.416,4.823800e+07,2.299,1.459,2.496000e+06,3.761000e+06,-3.889000e+06,2.898700e+07,24790.588,8.496
2,520263322,500004,198902,15184292,CLOUD ENGINEER,1464978601,13,3,2010.0,2015.0,NaN,100.0,NaN,10.0,1467492710,NaN,f,pg,125,30,80,5.0,"10,001+ employees",0,0,1,0,0,0,0,0,0,0,98117,55897,28341,16405,39,1,1139,1992,2.828000e+06,7.797,NaN,39.631,0.982,-13.416,4.823800e+07,2.299,1.459,2.496000e+06,3.761000e+06,-3.889000e+06,2.898700e+07,24790.588,8.496
3,514494796,500004,198902,16118626,CLOUD ENGINEER,1464978601,13,3,2010.0,2015.0,NaN,100.0,NaN,10.0,1465921088,NaN,f,pg,65,16,70,5.0,"10,001+ employees",0,0,1,0,0,0,0,0,0,0,112523,70630,33587,21011,81,1,1104,1994,2.828000e+06,7.797,NaN,39.631,0.982,-13.416,4.823800e+07,2.299,1.459,2.496000e+06,3.761000e+06,-3.889000e+06,2.898700e+07,24790.588,8.496
4,527199523,500004,198902,16429157,CLOUD ENGINEER,1464978601,13,3,2010.0,2015.0,NaN,100.0,NaN,10.0,1469211896,NaN,f,graduation,125,30,70,5.0,"10,001+ employees",0,0,1,0,0,0,0,0,0,0,98117,55897,28341,16405,39,1,1139,1995,2.828000e+06,7.797,NaN,39.631,0.982,-13.416,4.823800e+07,2.299,1.459,2.496000e+06,3.761000e+06,-3.889000e+06,2.898700e+07,24790.588,8.496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6315269,900035977,753520,507216,20880431,WEB DEVELOPER,1541744159,13,3,2016.0,2018.0,NaN,100.0,NaN,10.0,1541770440,NaN,a,graduation,65,16,95,NaN,51-200 employees,1,0,0,1,0,0,0,0,0,1,11045,7196,6546,4021,36,1,189,1996,-3.965100e+01,-17.473,NaN,NaN,0.528,-3.263,2.796117e+02,NaN,NaN,-4.922248e+01,-4.885564e+01,-5.607427e+00,1.718731e+02,NaN,NaN
6315270,900079862,753520,507216,18968531,WEB DEVELOPER,1541744159,13,3,2016.0,2018.0,NaN,100.0,NaN,10.0,1541847669,NaN,a,graduation,65,16,80,NaN,51-200 employees,0,0,0,1,0,0,0,0,0,1,11045,7196,6546,4021,36,1,189,1995,-3.965100e+01,-17.473,NaN,NaN,0.528,-3.263,2.796117e+02,NaN,NaN,-4.922248e+01,-4.885564e+01,-5.607427e+00,1.718731e+02,NaN,NaN
6315271,900102685,753520,507216,20629571,WEB DEVELOPER,1541744159,13,3,2016.0,2018.0,NaN,100.0,NaN,10.0,1541867265,NaN,f,graduation,160,34,60,NaN,51-200 employees,0,0,0,1,0,0,0,0,0,1,11045,7196,6546,4021,36,1,189,1996,-3.965100e+01,-17.473,NaN,NaN,0.528,-3.263,2.796117e+02,NaN,NaN,-4.922248e+01,-4.885564e+01,-5.607427e+00,1.718731e+02,NaN,NaN
6315272,899634404,753520,507216,20831934,WEB DEVELOPER,1541744159,13,3,2016.0,2018.0,NaN,100.0,NaN,10.0,154175671

In [50]:
# add "course_type" but no rows that are nan

df = df.dropna(subset=["course_type"])
# add important columns
df_new = df.filter(["date_of_posting", "job_role", "api_job_gender", "year_of_passing_from", "year_of_passing_to", "hq_marks_percentage_from", "hq_marks_cgpa_from", "api_applications_created", "FemaleFlag", "CalLetterFlag", "India_HQ", "yearBirth", "course_type", "no_of_employees"], axis=1)

# add "no_of_employees" but no rows that are nan
df_new["no_of_employees"] = df_new["no_of_employees"].fillna(1)

# drop birth years that are not in the range 1950-2007
df_new = df_new[(df_new['yearBirth'] >= 1950) & (df_new['yearBirth'] <= 2007)]

In [52]:
edu_level = {"graduation": 1, "pg": 2, "others_category": 0}
df_new["course_type"] = df_new["course_type"].map(edu_level)

In [54]:
# split data into pre and post mar 9 2017

df_pre = df_new[df_new["api_applications_created"] <= 1489039200]
df_post = df_new[df_new["api_applications_created"] > 1489039200]

In [58]:
# train the pre network

# split the data into train and test
pre_features = df_pre.drop("CalLetterFlag", axis=1)
pre_labels = df_pre["CalLetterFlag"]

n_samples, n_features = pre_features.shape

X_train, X_test, Y_train, Y_test = train_test_split(pre_features, pre_labels, test_size=0.2, random_state=42)


In [59]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
X_train = torch.from_numpy(X_train.astype('float32'))
X_test = torch.from_numpy(X_test.astype('float32'))
Y_train = torch.from_numpy(Y_train.values.astype('float32'))
Y_test = torch.from_numpy(Y_test.values.astype('float32'))